# Imports


In [61]:
import minari
import torch

In [60]:
dataset = minari.load_dataset("D4RL/pointmaze/large-v2")

# Prior Functionality

### Getting functions/methods from other classes

In [3]:
class TrajectorySet: 
    def __init__(self): 
        """
        trajectories: a dictionary housing all of the trajectories. The dictionary structure is: 
            {
                1: [trajectory, length of trajectory]
                2: [ ... ]
                etc...
            } 

        num_trajectories: the number of trajectories currently in the set. 
        """

        self.trajectories = {} 
        self.num_trajectories = 0 
    
    def add_trajectory(self, trajectory):
        self.trajectories[self.num_trajectories] = [trajectory, len(trajectory)] 
        self.num_trajectories += 1
    
    def get_num_trajectories(self):
        return self.num_trajectories

    def get_trajectory(self, index): 
        assert index < self.num_trajectories, "Specified index is too large."
        return self.trajectories[index]
    
    def get_trajectory_set(self): 
        return self.trajectories
    
    def get_total_states(self): 
        sum = 0
        for _, v in self.trajectories.items(): 
            sum += v[1]
        return sum 

    def generate_trajectories(self, n_trajectories: int = 2): 
        """
        Generates a specified number of trajectories and saves them into the TrajectorySet class. 

        This runs the scripted agent, where the agent uses a PD controller to follow a 
        path of waypoints generated with QIteration until it reaches the goal.
        """
        ep_data = dataset.sample_episodes(n_episodes=n_trajectories) # sample trajectories

        # adds all of the sampled trajectories into the TrajectorySet 
        for i in range(len(ep_data)):
            ep = ep_data[i] 

            # Note: only saving states since we only need state representations in the encoder 
            self.add_trajectory(ep.observations["observation"]) 

# Sampler

### Need to sample and get positive and negative pairs (states). 

In [190]:
class Sampler(): 
    def __init__(self, T: TrajectorySet, dist="g"): 
        """
        T: The Trajectory Set class 
        dist: The distribution used for centering over the anchor state. 
            ['u', 'g', 'l', 'e'] - uniform, gaussian, laplace, exponential
        """

        self.T = T 
        self.dist = dist

    def sample_anchor_state(self, t: list) -> tuple[list, int]: 
        """
        Given a trajectory, we sample the anchor state s_i uniformly. 

        Args: 
            t: The given trajectory we sample from. 

        Returns: 
            A tuple containing [s_i, idx]
            s_i: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
            idx: The time step of s_i. 
        """
        idx = torch.randint(low=0, high=len(t), size=(1,)).item()
        s_i = t[idx] 
        return [s_i, idx]

    def sample_positive_pair(self, t: list, anchor_state: tuple[list, int]) -> tuple[list, int]: 
        """
        Given the same trajectory that s_i was sampled from, 
        center a gaussian distribution around s_i to get obtain its positve pair: s_j. 
        
        Args: 
            t: The given trajectory, which must be the same as the trajectory that was used to sample the anchor state. 
            anchor_state: The anchor state; a tuple containing [s_i, idx].
            s_i: The state itself.
            idx: The time step of s_i.
            
        
        Return: 
            A tuple containing [s_j, idx]
            s_j: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
            idx: The time step of s_j.    
        """
        std = 15     # we use 15 to replicate the paper's hyperparams 
        b = 15       # laplace scale hyper param
        gamma = 0.99 # exponential hyper param 

        _, si_idx = anchor_state

        while True: 
            if self.dist == "u": 
                # uniform 
                sj_idx = torch.randint(low=0, high=len(t), size=(1,))
            elif self.dist == "g": 
                # gaussian 
                sj_idx = torch.normal(mean=si_idx, std=std, size=(1,))
            elif self.dist == "l": 
                # laplacian
                sj_idx = torch.distributions.laplace.Laplace(loc=si_idx, scale=b).sample() 
            elif self.dist == "e": 
                # exponential 
                i = int(torch.distributions.exponential.Exponential(rate=gamma).sample()) + 1   # +1 so we don't get an offset of 0
                sj_idx = si_idx + i 
            else: 
                # default to gaussian
                sj_idx = torch.normal(mean=si_idx, std=std, size=(1,))

            sj_idx = int(sj_idx) 

            # Ensures we don't choose an index out of range or the same state. 
            if (sj_idx < len(t)) and (sj_idx > 0) and (sj_idx != si_idx): 
                break 
        
        s_j = t[sj_idx] 

        return [s_j, sj_idx]
    
    def sample_batch(self, batch_size=1024, k=2) -> list[tuple]: 
        """ 
        Creates a batch of anchor states, their positive pairs, and negative pairs. 
        There will be 2(batch_size - 1) amount of negative examples per positive pair.

        Args: 
            T: The trajectory set class (must be empty). 
            batch_size: The size of the batch to be generated.
            k: A hyperparameter that dictates the average number of 
                positive pairs sampled from the same trajectory. The 
                lower the number, the lesser the chance of false negatives. 
        
        Returns: 
            A list of tuples containing the anchor_state and its positive pair. 
            The list is the same length as batch_size. 
        """ 

        batch = [] 

        # Generate trajectory set 
        n_trajectories = batch_size // k
        self.T.generate_trajectories(n_trajectories= n_trajectories)

        for _ in range(batch_size): 
            # Sample anchor state 
            rng = torch.randint(low=0, high=n_trajectories, size=(1,)).item() 
            t = self.T.get_trajectory(index=rng)[0]
            
            anchor_state = self.sample_anchor_state(t) 

            # Sample positive pair 
            positive_pair = self.sample_positive_pair(t, anchor_state=anchor_state)

            # Retrieve states; time-steps aren't necessary. 
            s_i = anchor_state[0]
            s_j = positive_pair[0]

            batch.append([s_i, s_j]) 

        return batch 

In [191]:
"""
TESTING CELL!
"""
T = TrajectorySet()

uniform = "u"
gaussian = "g"
laplacian = "l"
exponential = "e"

s = Sampler(T, laplacian)

T.generate_trajectories(n_trajectories=1) 
t = T.get_trajectory(0)[0] 
anchor_state = s.sample_anchor_state(t)
print(f"s_i: {anchor_state[0]}, at time step: {anchor_state[1]}")

positive_pair = s.sample_positive_pair(t=t, anchor_state=anchor_state)
print(f"s_j: {positive_pair[0]}, at time step: {positive_pair[1]}")


s_i: [ 1.23471106 -1.09534703 -4.19401611 -0.63667983], at time step: 121
s_j: [ 0.83586222 -1.15200978 -5.15372873 -0.51211443], at time step: 129


In [192]:
"""
TESTING CELL! 

Testing to see if sample_batch works properly. 
"""

batch = s.sample_batch(batch_size=128, k=2)
print("Batch:", batch)
len(batch)

Batch: [[array([-4.52341394,  1.80484523,  0.61485544,  3.60875914]), array([-4.5635801 ,  2.59930941, -0.06907109,  4.50902366])], [array([ 0.8095117 ,  2.99843348, -4.50602492,  0.15929856]), array([ 0.90113507,  2.99453386, -4.90616025,  0.43689374])], [array([ 4.37611381, -0.87506199, -0.02636711, -4.00345563]), array([ 4.39276279, -0.65457436, -0.65943704, -4.76022324])], [array([ 4.33297898,  2.80084401, -0.94105482,  2.12395715]), array([ 4.14279034,  2.90913259, -3.04424236,  0.61338801])], [array([ 4.51987751,  3.1810503 , -0.42885129, -0.0169915 ]), array([ 4.51143445,  3.17357244, -0.48931266, -0.49267112])], [array([4.41769864, 1.59456961, 0.31644512, 4.42240257]), array([4.39122444, 1.3565314 , 0.51495713, 4.90552071])], [array([-0.6521685 ,  0.94397283,  3.52559844, -0.56330294]), array([-0.8053618 ,  0.94845906,  2.8459734 ,  0.2650348 ])], [array([ 0.95183727, -1.09307095,  4.6484805 ,  0.39362756]), array([ 1.04427165, -1.08721839,  4.6720146 ,  0.2693501 ])], [array([

128